<a href="https://colab.research.google.com/github/DATA601-CustSegment-Nov2025-to-Feb2026/New-CustSegment-Project/blob/main/Customer_Segmentation_(Merging_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries & Load Datasets

In [ ]:
# Import necessary libraries
import pandas as pd
import dask.dataframe as dd

from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Paths to the 3 datasets stored in Google Drive
transaction_demographic_product_causal_df_path = '/content/drive/MyDrive/0. Colab Notebooks (DATA 60X Project)/transaction_demographic_product_causal_df.csv'
merged_coupon_df_path = '/content/drive/MyDrive/0. Colab Notebooks (DATA 60X Project)/merged_coupon_df.csv'
campaign_desc_file_path = '/content/drive/MyDrive/0. Colab Notebooks (DATA 60X Project)/campaign_desc.csv'


# Read the 3 datasets from their CSV files
transaction_demographic_product_causal_df = dd.read_csv(
    transaction_demographic_product_causal_df_path,
    dtype={
        'description': 'object',
        'display': 'object',
        'mailer': 'object'
    }
)

merged_coupon_df = dd.read_csv(
    merged_coupon_df_path,
    dtype={
        'campaign': 'float',
    }
)

# Explore Each Dataset: First 5 Rows & Dataset Dimensions

## 'transaction_demographic_product_causal' Dataset

In [ ]:
# Show all columns
pd.set_option('display.max_columns', None)

# Display first 5 rows of the dataset
transaction_demographic_product_causal_df.head(5)

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,description,campaign,start_day,end_day,received_campaign,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,display,mailer
0,459,32811565816,392,1039682,1,0.99,31762,0.00,1553,57,0.0,0.0,<NA>,NaN,NaN,NaN,no,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,PRODUCE,National,VEGETABLES SALAD,VARIETY LETTUCE,24 CT,<NA>,<NA>
1,97,32811576590,392,1060292,1,2.50,364,-0.69,1857,57,0.0,0.0,TypeB,4.0,372.0,404.0,yes,45-54,U,75-99K,Unknown,Single Female,1,None/Unknown,754,GROCERY,National,PASTA SAUCE,MAINSTREAM,45 OZ,<NA>,<NA>
2,878,32811571492,392,947201,1,2.99,297,-0.30,2002,57,0.0,0.0,TypeB,5.0,377.0,411.0,yes,25-34,U,35-49K,Unknown,1 Adult Kids,3,2,397,MEAT-PCKGD,National,FROZEN MEAT,FRZN BREADED PREPARED CHICK,11.5 OZ,<NA>,<NA>
3,1453,32811562421,392,1139830,1,0.39,31862,0.00,2059,57,0.0,0.0,<NA>,NaN,NaN,NaN,no,45-54,A,125-149K,Homeowner,2 Adults Kids,3,1,678,PASTRY,National,BREAD,BREAD:SAVORY/SPECIALTY,2 OZ,<NA>,<NA>
4,97,32811576590,392,874061,1,1.00,364,-0.32,1857,57,0.0,0.0,TypeB,4.0,372.0,404.0,yes,45-54,U,75-99K,Unknown,Single Female,1,None/Unknown,69,GROCERY,Private,REFRGRATD DOUGH PRODUCTS,REFRIGERATED BISCUITS REGULAR,16 OZ,7,A


In [ ]:
# Calculate the number of rows and columns in the Dask dataFrame
num_rows_transaction_demographic_product_causal_df = transaction_demographic_product_causal_df.shape[0].compute()
num_cols_transaction_demographic_product_causal_df = len(transaction_demographic_product_causal_df.columns)

print("Number of rows:", num_rows_transaction_demographic_product_causal_df)
print("Number of columns:", num_cols_transaction_demographic_product_causal_df)

Number of rows: 2595732
Number of columns: 32


## 'merged_coupon_df' Dataset

In [ ]:
# Display first 5 rows of the dataset
merged_coupon_df.head()

,coupon_upc,product_id,campaign,household_key,day
0,10000089061,27160,4.0,NaN,NaN
1,10000089064,27754,9.0,321.0,446.0
2,10000089064,27754,9.0,1773.0,439.0
3,10000089073,28897,12.0,1229.0,491.0
4,10000089073,28897,12.0,1633.0,497.0


In [ ]:
# Convert 'household_key' and 'day' columns to nullable integers (Int64)
merged_coupon_df['household_key'] = merged_coupon_df['household_key'].astype('Int64')
merged_coupon_df['day'] = merged_coupon_df['day'].astype('Int64')

# Display first 5 rows of the dataset after conversion
merged_coupon_df.head()

,coupon_upc,product_id,campaign,household_key,day
0,10000089061,27160,4.0,<NA>,<NA>
1,10000089064,27754,9.0,321,446
2,10000089064,27754,9.0,1773,439
3,10000089073,28897,12.0,1229,491
4,10000089073,28897,12.0,1633,497


In [ ]:
# Calculate the number of rows and columns in the Dask dataFrame
num_rows_merged_coupon_df = merged_coupon_df.shape[0].compute()
num_cols_merged_coupon_df = len(merged_coupon_df.columns)

print("Number of rows:", num_rows_merged_coupon_df)
print("Number of columns:", num_cols_merged_coupon_df)

Number of rows: 2218551
Number of columns: 5


# Joining Dataset

## Join Merged Dataset (Transaction, Demographic, Product & Causal) With Merged Coupon Dataset

In [ ]:
# Join 'transaction_demographic_product_causal_df' with 'merged_coupon_df' on household_key, day and product_id.
merged_transaction_coupon_df = transaction_demographic_product_causal_df.merge(merged_coupon_df, on=['household_key', 'day', 'product_id', 'campaign'], how='left')

# Persist in memory
merged_transaction_coupon_df = merged_transaction_coupon_df.persist()

/usr/local/lib/python3.12/dist-packages/dask/dataframe/multi.py:169: UserWarning: Merging dataframes with merge column data type mismatches: 
+------------------------------------+------------+-------------+
| Merge columns                      | left dtype | right dtype |
+------------------------------------+------------+-------------+
| ('household_key', 'household_key') | int64      | Int64       |
| ('day', 'day')                     | int64      | Int64       |
+------------------------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


In [ ]:
# Show all columns
pd.set_option('display.max_columns', None)

# Display first 5 rows of the merged dataset
merged_transaction_coupon_df.head(5)

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,description,campaign,start_day,end_day,received_campaign,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,display,mailer,coupon_upc
0,459,32811565816,392,1039682,1,0.99,31762,0.00,1553,57,0.0,0.0,<NA>,NaN,NaN,NaN,no,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2,PRODUCE,National,VEGETABLES SALAD,VARIETY LETTUCE,24 CT,<NA>,<NA>,NaN
1,97,32811576590,392,1060292,1,2.50,364,-0.69,1857,57,0.0,0.0,TypeB,4.0,372.0,404.0,yes,45-54,U,75-99K,Unknown,Single Female,1,None/Unknown,754,GROCERY,National,PASTA SAUCE,MAINSTREAM,45 OZ,<NA>,<NA>,NaN
2,878,32811571492,392,947201,1,2.99,297,-0.30,2002,57,0.0,0.0,TypeB,5.0,377.0,411.0,yes,25-34,U,35-49K,Unknown,1 Adult Kids,3,2,397,MEAT-PCKGD,National,FROZEN MEAT,FRZN BREADED PREPARED CHICK,11.5 OZ,<NA>,<NA>,NaN
3,1453,32811562421,392,1139830,1,0.39,31862,0.00,2059,57,0.0,0.0,<NA>,NaN,NaN,NaN,no,45-54,A,125-149K,Homeowner,2 Adults Kids,3,1,678,PASTRY,National,BREAD,BREAD:SAVORY/SPECIALTY,2 OZ,<NA>,<NA>,NaN
4,97,32811576590,392,874061,1,1.00,364,-0.32,1857,57,0.0,0.0,TypeB,4.0,372.0,404.0,yes,45-54,U,75-99K,Unknown,Single Female,1,None/Unknown,69,GROCERY,Private,REFRGRATD DOUGH PRODUCTS,REFRIGERATED BISCUITS REGULAR,16 OZ,7,A,NaN


In [ ]:
# Calculate the number of rows and columns in the Dask dataFrame
num_rows_merged_transaction_coupon_df = merged_transaction_coupon_df.shape[0].compute()
num_cols_merged_transaction_coupon_df = len(merged_transaction_coupon_df.columns)

print("Number of rows in merged_transaction_coupon_df:", num_rows_merged_transaction_coupon_df)
print("Number of columns in merged_transaction_coupon_df:", num_cols_merged_transaction_coupon_df)

Number of rows in merged_transaction_coupon_df: 2595873
Number of columns in merged_transaction_coupon_df: 33


In [ ]:
# Calculate the number of rows and columns in the Dask dataFrame
num_rows_transaction_demographic_product_causal_df = transaction_demographic_product_causal_df.shape[0].compute()
num_cols_transaction_demographic_product_causal_df = len(transaction_demographic_product_causal_df.columns)

print("Number of rows in transaction_demographic_product_causal_df:", num_rows_transaction_demographic_product_causal_df)
print("Number of columns in transaction_demographic_product_causal_df:", num_cols_transaction_demographic_product_causal_df)

Number of rows in transaction_demographic_product_causal_df: 2595732
Number of columns in transaction_demographic_product_causal_df: 32


In [ ]:
# Calculate the number of row difference between 'transaction_demographic_product_causal_df' and 'merged_transaction_coupon_df'
row_difference_2 = num_rows_merged_transaction_coupon_df - num_rows_transaction_demographic_product_causal_df

print(f"Row difference between 'transaction_demographic_product_causal_df' and 'merged_transaction_coupon_df': {row_difference_2} rows")

Row difference between 'transaction_demographic_product_causal_df' and 'merged_transaction_coupon_df': 141 rows


In [ ]:
# List all columns except 'display' and 'mailer'
cols_to_check = [col for col in merged_transaction_coupon_df.columns
                 if col not in ['coupon_upc', 'campaign', 'household_key']]

# Create a boolean column marking duplicates within each partition
def mark_duplicates(df, subset):
    return df[df.duplicated(subset=subset, keep=False)]

# Apply to each partition
duplicates_merged_transaction_coupon_df = merged_transaction_coupon_df.map_partitions(
    mark_duplicates, subset=cols_to_check
)

# Compute number of duplicate rows
num_duplicates_merged_transaction_coupon_df = len(duplicates_merged_transaction_coupon_df.compute())

print(f"Number of duplicate rows: {num_duplicates_merged_transaction_coupon_df}")

Number of duplicate rows: 231


In [ ]:
# Convert 'coupon_upc' column from gloat to string to avoid scientific notation caused by large numeric ID.
duplicates_merged_transaction_coupon_df['coupon_upc'] = duplicates_merged_transaction_coupon_df['coupon_upc'].astype('str')

# Display first 5 rows of the merged dataset
duplicates_merged_transaction_coupon_df.head(5)

# After the left join, the the numner of rows increased from 2,596,590 to 2,596,722 (182 rows).
# This is due to multiple matching rows for 'coupon_upc' and 'campaign' columns in the right table (merged_transaction_coupon_df).

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,description,campaign,start_day,end_day,received_campaign,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,display,mailer,coupon_upc
45754,573,33118502623,415,849697,1,2.99,445,-0.50,2152,60,-1.0,0.0,TypeA,8.0,412.0,460.0,yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1225,GROCERY,National,FLUID MILK PRODUCTS,REFRIGERATED COFFEE CREAMERS,32 OZ,<NA>,<NA>,10000085364.0
45755,573,33118502623,415,849697,1,2.99,445,-0.50,2152,60,-1.0,0.0,TypeA,8.0,412.0,460.0,yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1225,GROCERY,National,FLUID MILK PRODUCTS,REFRIGERATED COFFEE CREAMERS,32 OZ,<NA>,<NA>,55000013033.0
50420,1098,33132674533,416,920710,1,3.74,343,-1.25,1849,60,0.0,0.0,TypeA,8.0,412.0,460.0,yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4971,DRUG GM,National,FIRST AID PRODUCTS,FIRST AID MEDICATIONS,,<NA>,<NA>,50008130078.0
50421,1098,33132674533,416,920710,1,3.74,343,-1.25,1849,60,0.0,0.0,TypeA,8.0,412.0,460.0,yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4971,DRUG GM,National,FIRST AID PRODUCTS,FIRST AID MEDICATIONS,,<NA>,<NA>,50008130078.0
53992,1618,33155490849,418,1059067,2,13.96,406,-2.02,2030,60,0.0,0.0,TypeA,8.0,412.0,460.0,yes,35-44,A,75-99K,Homeowner,Single Female,2,None/Unknown,5037,DRUG GM,National,ANALGESICS,ADULT ANALGESICS,,<NA>,<NA>,52586611376.0


In [ ]:
# As our aim is to focus on transactional-level patterns for customer segmentation, the extra rows don't represent new purchases.
# They exist because a single transaction matched multiple coupons or campaigns (a customer may have used or been eligible for multiple coupons or campaigns on the same product purchase.)
# Keeping these duplicates could inflate the number of transactions and bias our customer segmnetation, making it look like some customers buy more than they actually do.

# Define columns to check for duplicates (excluding 'coupon_upc' and 'campaign')
cols_to_identify_merged_transaction_coupon_df = [
    'household_key', 'basket_id', 'day', 'product_id', 'quantity',
    'sales_value', 'store_id', 'retail_disc', 'trans_time', 'week_no', 'coupon_disc',
    'coupon_match_disc', 'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
    'hh_comp_desc', 'household_size_desc', 'kid_category_desc',
    'manufacturer', 'department', 'brand', 'commodity_desc',
    'sub_commodity_desc', 'curr_size_of_product', 'display', 'mailer', 'received_campaign'
]

# We want to keep the coupon with the highest dicount
# Sort by discount descending and keep first per transaction
merged_transaction_coupon_df = merged_transaction_coupon_df.sort_values('coupon_disc', ascending=True)
merged_transaction_coupon_df_no_dup = merged_transaction_coupon_df.drop_duplicates(subset=cols_to_identify_merged_transaction_coupon_df , keep='first')

# Persist in memory
merged_transaction_coupon_df_no_dup = merged_transaction_coupon_df_no_dup.persist()

# Display first 5 rows of the merged dataset
merged_transaction_coupon_df_no_dup.head(5)

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,description,campaign,start_day,end_day,received_campaign,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,display,mailer,coupon_upc
10289,979,40765361731,606,5978656,0,0.00,364,0.0,1621,87,-21.49,0.0,TypeA,18.0,587.0,642.0,yes,35-44,B,35-49K,Homeowner,1 Adult Kids,2,1,1,,National,,,,<NA>,<NA>,NaN
165038,1463,30822740847,256,864900,1,19.99,372,0.0,2006,37,-19.99,0.0,TypeA,26.0,224.0,264.0,yes,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,764,GROCERY,National,HOUSEHOLD CLEANG NEEDS,TOOLS - FLOOR & FURNITURE,1 KIT,<NA>,<NA>,NaN
251258,1463,27609884211,53,5978656,0,0.00,372,0.0,1908,8,-16.99,0.0,<NA>,NaN,NaN,NaN,no,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,,National,,,,<NA>,<NA>,NaN
315399,1228,31883657167,328,900379,2,24.38,406,-1.6,1438,48,-16.98,0.0,TypeA,30.0,323.0,369.0,yes,45-54,U,100-124K,Unknown,Single Female,1,None/Unknown,69,SEAFOOD-PCKGD,Private,SEAFOOD - FROZEN,SEAFOOD-FRZ-IQF RAW SHRIMP RW,16 OZ,0,A,NaN
309183,17,28179613679,100,997415,4,103.32,361,0.0,1741,15,-16.00,0.0,<NA>,NaN,NaN,NaN,no,65+,B,Under 15K,Homeowner,2 Adults No Kids,2,None/Unknown,539,DRUG GM,National,CIGARETTES,CIGARETTES,500843 CTN,<NA>,<NA>,NaN


In [ ]:
# Calculate the number of rows and columns in the Dask dataFrame
num_rows_merged_transaction_coupon_df_no_dup = merged_transaction_coupon_df_no_dup.shape[0].compute()
num_cols_merged_transaction_coupon_df_no_dup = len(merged_transaction_coupon_df_no_dup.columns)

print("Number of rows in merged_transaction_coupon_df:", num_rows_merged_transaction_coupon_df_no_dup)
print("Number of columns in merged_transaction_coupon_df:", num_cols_merged_transaction_coupon_df_no_dup)

Number of rows in merged_transaction_coupon_df: 2595732
Number of columns in merged_transaction_coupon_df: 33


In [ ]:
# Save 'final_merged_df' as CSV file
merged_transaction_coupon_df_no_dup.compute().to_csv('/content/drive/MyDrive/0. Colab Notebooks (DATA 60X Project)/customer_segmentation_after_merging.csv', index=False)